# 自然言語処理入門

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_5/intro-to-nlp.ipynb)

テキストデータの処理と分析の基礎である自然言語処理を概観しましょう。

まず、テキストデータの前処理を段階を踏んで見ていきます。次に、自然言語処理の基本的な考え方とテキストデータのベクトル化を学びます。

## テキストデータの前処理

テキストデータを分析する際には、基本的に以下の処理が行われます。

1. 形態素解析
    * 単語分割
    * 品詞付与
    * 原形抽出
2. 構文解析


形態素は、「言葉が意味を持つまとまりの単語の最小単位」で、形態素解析は、文章を一つ一つの形態素に分ける技術です。単語が区切られていない日本語などの言語では特に重要です。

日本語を例に、`spaCy`と`GiNZA`というライブラリを使って処理の過程を見ていきましょう。

`spaCy`では上の一連の処理をまとめて行ってくれます。

テキストデータとして、オー・ヘンリー（結城浩訳）[『最後の一枚の葉』](https://www.hyuki.com/trans/leaf.html)の冒頭部分を使います。

In [ ]:
text = """
ワシントン・スクエア西にある小地区は、 道路が狂ったように入り組んでおり、 「プレース」と呼ばれる区域に小さく分かれておりました。 この「プレース」は不可思議な角度と曲線を描いており、 一、二回自分自身と交差している通りがあるほどでした。 かつて、ある画家は、この通りが貴重な可能性を持っていることを発見しました。 例えば絵や紙やキャンバスの請求書を手にした取り立て屋を考えてみてください。 取り立て屋は、この道を歩き回ったあげく、 ぐるりと元のところまで戻ってくるに違いありません。 一セントも取り立てることができずにね。
それで、芸術家たちはまもなく、奇妙で古いグリニッチ・ヴィレッジへとやってきました。 そして、北向きの窓と十八世紀の切り妻とオランダ風の屋根裏部屋と安い賃貸料を探してうろついたのです。 やがて、彼らは しろめ製のマグやこんろ付き卓上なべを一、二個、六番街から持ち込み、 「コロニー」を形成することになりました。
ずんぐりした三階建ての煉瓦造りの最上階では、スーとジョンジーがアトリエを持っていました。 「ジョンジー」はジョアンナの愛称です。 スーはメイン州の、ジョンジーはカリフォルニア州の出身でした。 二人は八番街の「デルモニコの店」の定食で出会い、 芸術と、チコリーのサラダと、ビショップ・スリーブの趣味がぴったりだとわかって、 共同のアトリエを持つことになったのでした。
"""

In [ ]:
# ライブラリのインストール（初回のみ）

!pip install spacy ginza ja-ginza
!pip install scikit-learn pandas

In [ ]:
# ライブラリのインポート
import spacy

# 日本語モデルのロード
nlp = spacy.load("ja_ginza")

# 解析
doc = nlp(text)

# 結果の確認
for token in doc:
    print(token)

形態素解析の結果には、語の原形や品詞の情報も含まれます。

In [ ]:
for token in doc:
    print(f"{token}\t{token.lemma_}\t{token.pos_}\t{token.tag_}")

構文解析の結果は、次のように確認できます。

In [ ]:
import pandas as pd

# 解析結果をpandasのDataFrameに入れる

df = pd.DataFrame({
    "text": token.text,
    "lemma_": token.lemma_,
    "pos_": token.pos_,
    "tag_": token.tag_,
    "dep_": token.dep_,
    "children": list(token.children)
} for token in doc)

In [ ]:
df

In [ ]:
# 係り受けの図を表示する

spacy.displacy.render(doc, style="dep", jupyter=True)

前処理が完了したところで、データの確認として単語の使用頻度を数えてみましょう。

In [ ]:
from collections import Counter

# 単語の頻度を数える
counter = Counter(token.lemma_ for token in doc)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

句読点や助詞など、意味がなさそうな言葉ばかりです。

より意味がある語を取り出すために、分析対象とする品詞を指定しましょう。具体的には、内容語である名詞、動詞、形容詞（、固有名詞）を指定すればよいでしょう。

In [ ]:
# 分析対象とする品詞の指定
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc if token.pos_ in include_pos)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ちょっとよくなりました。でも、「こと」「ある」「いる」などの一般的な名詞や動詞が多いように思えます。

これらを不要語として指定し、除去しましょう。

In [ ]:
# 分析対象とする品詞と不要語（ストップワード）を指定する
include_pos = ("NOUN", "VERB", "ADJ", "PROPN")
stopwords = ("する", "ある", "おる", "ない", "いう", "もの", "こと", "よう", "なる", "ほう", "いる", "くる")

# 再度単語の頻度を数える
counter = Counter(token.lemma_ for token in doc
                  if token.pos_ in include_pos and token.lemma_ not in stopwords)

# 出現頻度top 20を出力する
for word, count in counter.most_common(20):
    print(f"{count:>5} {word}")

ずっと良くなりました。

これは、語の出現頻度を要素としたテキストデータのベクトル表現で、Bag of Wordsと呼ばれます。

### tf-idf

Bag of Wordsは、「多く出現する語ほど重要である」という直観的な考え方に基づいてテキストデータの特徴を表現したベクトルであり、文書分類を含む多くの自然言語処理タスクに有用です。

しかし、いろいろな文書に多く出現すると予測される「わたし」などの言葉は、出現頻度のわりにさほど重要ではないと考えられます。今回の処理のようにこのような言葉を不要語として指定し、分析から除外することもできますが、この点を考慮した単語の重要度の指標として、tf-idfがあります。

tf-idfは、term frequency（単語頻度）-inverse document frequency（逆文書頻度）の略です。具体的には、「ある文書内である単語がどれくらい多い頻度で出現するか」を表すterm frequencyと、「全文書内である単語を含む文書がどれくらい少ない頻度で出現するか」を表すinverse document frequencyをかけ合わせた値です。どの文書にもよく出てくる単語の重要度を下げて、あまり出てこない単語の重要度を上げるために工夫された指標です。

$$
tf-idf = tf（単語頻度） \times \frac{N（文書総数）}{n（単語が出現する文書数）}
$$



tf-idfはヒューリスティックな指標で理論的な根拠はあまりありませんが、重要語を上手く抽出でき、文書を特徴づけることができることが知られています。

そのため、頻度の代わりにtf-idfを要素としてテキストデータをベクトル化することがよく行われています。



---

## テキストデータのベクトル化

以下の内容は斎藤（2018）『ゼロから作る Deep Learning 2』に基づいています。

日本語や英語など、私たちが普段使っている言葉を自然言語（Natural Language）と言います。自然言語処理（Natural Language Processing）とは、自然言語を処理する分野です。

自然言語処理の目標は、人の話す言葉をコンピュータに理解させ、私たちにとって役に立つことをコンピュータに行わせることです。
私たちの言葉は「文字」によって表現することができます。そして、言葉の意味は「単語」（正確には形態素）によって構成されます。そのため、自然言語をコンピュータに理解させるためには、「単語の意味」を理解させることが重要です。

ここでは、単語の意味を数値で表現する手法を学びます。

自然言語処理の研究や応用のために目的をもって収集されたテキストデータを「コーパス」と呼びます。WikipediaやGoogle Newsなどのテキストデータや、シェイクスピアや夏目漱石などの作品群もコーパスです。

コーパスはテキストデータであり、そこに含まれる文章は人によって書かれたものです。これはつまり、コーパスには自然言語に対する人の知識が含まれているということです。
文章の書き方、単語の選び方、単語の意味などがコーパスには含まれています。

テキストデータのベクトル化の目的は、人の知識が詰まったコーパスから自動的に効率よく、そのエッセンスを抽出することです。

## 簡単なコーパスの前処理

テキストデータを単語に分割し、分割した単語を単語IDのリストへ変換することで、データの前処理を行いましょう。

先程モジュールを使って自動で行った単語分割を手動で行います。

In [ ]:
text = "You say goodbye and I say hello."

In [ ]:
# 小文字に変換
text = text.lower()
text

In [ ]:
# 句点の前にスペースを挿入
text = text.replace(".", " .")
text

In [ ]:
# 文を単語に分割する
words = text.split(' ')
words

In [ ]:
# 単語のIDと単語の対応表を作る

word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [ ]:
word_to_id

In [ ]:
id_to_word

この2つの辞書を使えば、単語から単語IDの検索と、単語IDから単語の検索ができます。

In [ ]:
id_to_word[2]

In [ ]:
word_to_id["i"]

最後に、単語のリストを単語IDのリストに変換し、NumPy配列に変換します。

In [ ]:
import numpy as np

corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

これでコーパスの前処理は終了です。

In [ ]:
# 以上の処理を関数として実装する

def preprocess(text):
    text = text.lower()
    text = text.replace(".", " .")
    text = text.replace(",", " ,")
    text = text.replace("!", " !")
    text = text.replace("?", " ?")
    text = text.replace(";", " ;")
    text = text.replace(":", " :")
    text = text.replace("\"", "")
    text = text.replace("\'", "")
    text = text.replace("\n", "")
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

## 単語の分散表現


次に、コーパスを使って単語の意味を抽出しましょう。具体的には、単語をベクトルで表すことを目指します。これは、自然言語処理の分野では、単語の分散表現と呼ばれます。

単語の分散表現に関する手法は、「単語の意味は、周囲の単語によって形成される」というアイデアに基づいています。これは、分布仮説と呼ばれるものです。

分布仮説では、単語自体には意味がなく、その単語の「コンテキスト（文脈）」によって、単語の意味が形成されると言われています。

たしかに、意味的に同じ単語は、同じような文脈で多く出現します。

例えば、

* I drink beer.
* We drink wine.

のようにdrinkの近くには飲み物があらわれやすいでしょう。


* I guzzle beer.
* We guzzle wine.

のような文章では、guzzleという単語がdrinkと同じような文脈で使われていることが分かります。そして、guzzleとdrinkが近い意味の単語だということが導けます。

## 共起行列

分布仮説に基づいて、単語をベクトルで表す方法を考えます。

素直な方法は、周囲の単語を数えることです。

さきほど用意したコーパスに含まれるそれぞれの単語について、そのコンテキスト（目当ての単語の周囲）に含まれる単語の頻度を数えていきます。

例えば、youという単語に着目すると、

||you|say|goodbye|and|i|hello|.|
|---|---|---|---|---|---|---|---|
|you|0|1|0|0|0|0|0|

このような表現になります。

全ての語に対してこれを数えると、

||you|say|goodbye|and|i|hello|.|
|---|---|---|---|---|---|---|---|
|you|0|1|0|0|0|0|0|
|say|1|0|1|0|1|1|0|
|goodbye|0|1|0|1|0|0|0|
|and|0|0|1|0|1|0|0|
|i|0|1|0|1|0|0|0|
|hello|0|1|0|0|0|0|1|
|.|0|0|0|0|0|1|0|

となります。

これをNumPy配列にすることで、共起行列ができます。この共起行列を使うと、各単語の分散表現が求められます。

In [ ]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
])

In [ ]:
# youの分散表現
print(id_to_word[0], C[0])

In [ ]:
# andの分散表現
print(id_to_word[3], C[3])

In [ ]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

単語の分散表現を使うことで、単語の類似度を計算するなど、より高度な処理ができるようになります。

## ベクトル間の類似度

単語のベクトル表現の類似度の指標としては、コサイン類似度がよく使われます。コサイン類似度は、$v = (v_1, v_2, v_3, ..., v_n)$と$w = (w_1, w_2, w_3, ..., w_n)$の2つのベクトルがあるとき、次のように定義されます。

$$
cos(v, w) = \frac{v \cdot w}{|v||w|} = \frac{x_{1}y_{1} + \cdots + x_{n}y_{n}}{\sqrt{x_1^2 + \cdots + x_n^2}\sqrt{y_1^2 + \cdots + y_n^2}}
$$

コサイン類似度は、直感的には「2つのベクトルがどれだけ同じ方向を向いているか」を表します。2つのベクトルが完全に同じ方向を向いているときコサイン類似度は1になり、完全に逆向きだと-1になります。

コサイン類似度を実装すると、次のようになります。ただ、ゼロベクトル（ベクトルの要素がすべて0のベクトル）が引数に入ると「0除算」が発生してしまいます。

この問題には、除算を行う際に小さな値（eps）を加算することで対応します。デフォルト値として1e-8（0.00000001）を設定しましたが、これぐらい小さな値であれば、通常は浮動小数点の「丸め誤差」により他の値に吸収され、最終的な計算結果には影響を与えません。

In [ ]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

この関数を用いると、単語ベクトルの類似度を次のように求めることができます。

In [ ]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [ ]:
c0 = C[word_to_id['you']] # youのベクトル
c1 = C[word_to_id['i']] # iのベクトル
c2 = C[word_to_id['hello']] # helloのベクトル

In [ ]:
cos_similarity(c0, c1)

In [ ]:
cos_similarity(c1, c2)

上記結果から、youとiのコサイン類似度は0.70...であり、iとhelloのコサイン類似度は0.49...であることが分かりました。

iはhelloよりyouとより近い語であると言えそうです。

## 類似単語のランキング表示

コサイン類似度を使って、便利な関数を実装しましょう。ある単語がクエリとして与えられたときに、そのクエリに対して類似した単語を上位から順に表示する関数です。

In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):

    # クエリを取り出す
    if query not in word_to_id:
        print(f"{query} is not found")
        return

    print(f"\n[query] {query}")
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # コサイン類似度の算出
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # コサイン類似度の結果から、その値を高い順に出力
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(f"{id_to_word[i]}: {similarity[i]}")

        count += 1
        if count >= top:
            return

In [ ]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [ ]:
most_similar('you', word_to_id, id_to_word, C, top=5)

これを使うと、youに近い語が上から順番に出力されます。

iとyouは共に人称代名詞なので類似しているのは納得できますが、helloやgoodbyeとの類似度が高いのは不思議です。実はこれはコーパスのサイズが極端に小さいことに起因します。

もう少し大きいコーパスを使ってこの関数を試してみましょう。

## 少し大きいコーパスで試してみる

In [ ]:
text = """
In a little district west of Washington Square the streets have run crazy and broken themselves into small strips called "places." These "places" make strange angles and curves. One Street crosses itself a time or two. An artist once discovered a valuable possibility in this street. Suppose a collector with a bill for paints, paper and canvas should, in traversing this route, suddenly meet himself coming back, without a cent having been paid on account!

So, to quaint old Greenwich Village the art people soon came prowling, hunting for north windows and eighteenth-century gables and Dutch attics and low rents. Then they imported some pewter mugs and a chafing dish or two from Sixth Avenue, and became a "colony."

At the top of a squatty, three-story brick Sue and Johnsy had their studio. "Johnsy" was familiar for Joanna. One was from Maine; the other from California. They had met at the table d'hôte of an Eighth Street "Delmonico's," and found their tastes in art, chicory salad and bishop sleeves so congenial that the joint studio resulted.

That was in May. In November a cold, unseen stranger, whom the doctors called Pneumonia, stalked about the colony, touching one here and there with his icy fingers. Over on the east side this ravager strode boldly, smiting his victims by scores, but his feet trod slowly through the maze of the narrow and moss-grown "places."

Mr. Pneumonia was not what you would call a chivalric old gentleman. A mite of a little woman with blood thinned by California zephyrs was hardly fair game for the red-fisted, short-breathed old duffer. But Johnsy he smote; and she lay, scarcely moving, on her painted iron bedstead, looking through the small Dutch window-panes at the blank side of the next brick house.

One morning the busy doctor invited Sue into the hallway with a shaggy, grey eyebrow.

"She has one chance in - let us say, ten," he said, as he shook down the mercury in his clinical thermometer. " And that chance is for her to want to live. This way people have of lining-u on the side of the undertaker makes the entire pharmacopoeia look silly. Your little lady has made up her mind that she's not going to get well. Has she anything on her mind?"

"She - she wanted to paint the Bay of Naples some day." said Sue.

"Paint? - bosh! Has she anything on her mind worth thinking twice - a man for instance?"

"A man?" said Sue, with a jew's-harp twang in her voice. "Is a man worth - but, no, doctor; there is nothing of the kind."

"Well, it is the weakness, then," said the doctor. "I will do all that science, so far as it may filter through my efforts, can accomplish. But whenever my patient begins to count the carriages in her funeral procession I subtract 50 per cent from the curative power of medicines. If you will get her to ask one question about the new winter styles in cloak sleeves I will promise you a one-in-five chance for her, instead of one in ten."

After the doctor had gone Sue went into the workroom and cried a Japanese napkin to a pulp. Then she swaggered into Johnsy's room with her drawing board, whistling ragtime.

Johnsy lay, scarcely making a ripple under the bedclothes, with her face toward the window. Sue stopped whistling, thinking she was asleep.

She arranged her board and began a pen-and-ink drawing to illustrate a magazine story. Young artists must pave their way to Art by drawing pictures for magazine stories that young authors write to pave their way to Literature.

As Sue was sketching a pair of elegant horseshow riding trousers and a monocle of the figure of the hero, an Idaho cowboy, she heard a low sound, several times repeated. She went quickly to the bedside.

Johnsy's eyes were open wide. She was looking out the window and counting - counting backward.

"Twelve," she said, and little later "eleven"; and then "ten," and "nine"; and then "eight" and "seven", almost together.

Sue look solicitously out of the window. What was there to count? There was only a bare, dreary yard to be seen, and the blank side of the brick house twenty feet away. An old, old ivy vine, gnarled and decayed at the roots, climbed half way up the brick wall. The cold breath of autumn had stricken its leaves from the vine until its skeleton branches clung, almost bare, to the crumbling bricks.
"""

In [ ]:
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [ ]:
most_similar('say', word_to_id, id_to_word, C, top=5)